#### Pull tables to staging schema

In [ ]:
Countries_df = Countries_df.merge(Regions_df, left_on="Regions", right_on="Regions")
Countries_df = Countries_df.drop(columns=["Regions"])

In [ ]:
Country_to_recommendations = df3[['Countries Concerned','UPR Recommending States']]
Country_to_recommendations.index = Country_to_recommendations.index.set_names(['Recommendation_id'])
Country_to_recommendations = Country_to_recommendations.reset_index(level=0).drop_duplicates(subset='Recommendation_id', keep='first')
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='Countries Concerned', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['Countries Concerned','Country name']).rename(columns={"Country id": "Countries Concerned"})
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='UPR Recommending States', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['UPR Recommending States','Country name']).rename(columns={"Country id": "UPR Recommending States"})

In [ ]:
Recommendation_to_theme = UPR_cycle_cleaned[['Themes']]
Recommendation_to_theme.index = Recommendation_to_theme.index.set_names(['Recommendation id'])
Recommendation_to_theme = Recommendation_to_theme.reset_index(level=0)

In [ ]:
Recommendation_to_theme = Recommendation_to_theme.merge(Themes_df, left_on='Themes', right_on='Themes')
Recommendation_to_theme = Recommendation_to_theme.drop(columns=['Themes']).rename(columns={'Theme id':'Themes'})

In [ ]:
tags_to_themes = pd.melt(tags_matrix, value_vars=tags_matrix.columns.tolist(), var_name='Tag', value_name='Theme')
tags_to_themes = tags_to_themes.merge(tags_df[["Tags","Tag id"]], left_on='Tag', right_on='Tags').drop(columns=['Tags', 'Tag'])
tags_to_themes = tags_to_themes.merge(Themes_df[["Themes","Theme id"]], left_on='Theme', right_on='Themes').drop(columns=['Themes', 'Theme'])

### Normalized join table Dataframes

In [ ]:
Country_to_recommendations

In [ ]:
Recommendation_to_theme

In [ ]:
tags_to_themes

#### Upload tables to Production schema